
# Predição de Sobrevivência de Passageiros do Titanic usando Naive Bayes

Este notebook irá demonstrar como aplicar o algoritmo de Naive Bayes para prever se um passageiro do Titanic sobreviveu ou não, com base em um conjunto de características.

Vamos entender passo a passo como preparar os dados, treinar o modelo, e fazer previsões. Além disso, explicaremos a teoria por trás do Naive Bayes.

## O que é o Algoritmo de Naive Bayes?

Naive Bayes é um algoritmo de classificação baseado no teorema de Bayes, que assume que as características são independentes umas das outras. Ele é chamado de "ingênuo" (naive) porque essa suposição raramente é verdadeira na prática, mas o algoritmo ainda pode funcionar bem em muitos cenários.

O teorema de Bayes é dado por:

$$ P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)} $$

Onde:

- **P(A|B)** é a probabilidade de A acontecer dado que B aconteceu.
- **P(B|A)** é a probabilidade de B acontecer dado que A aconteceu.
- **P(A)** é a probabilidade de A acontecer.
- **P(B)** é a probabilidade de B acontecer.

O algoritmo Naive Bayes usa essa fórmula para calcular a probabilidade de uma amostra pertencer a uma determinada classe.

#### Por que Gaussiano?
A versão gaussiana do Naive Bayes é usada quando as características contínuas (como idade, preço da passagem, etc.) são normalmente distribuídas. Isso significa que, ao invés de categorizar os dados, o algoritmo assume que cada característica segue uma **distribuição normal** (ou Gaussiana) e calcula a probabilidade com base nessa distribuição. Assim, para uma dada característica \( x \), sua probabilidade é calculada pela função de densidade gaussiana:

$$
P(x|A) = \frac{1}{\sqrt{2 \pi \sigma_A^2}} \exp\left( -\frac{(x - \mu_A)^2}{2 \sigma_A^2} \right)
$$

Onde:
- $\mu_A$ é a média da característica $x$ para a classe  $A$ (sobreviveu ou não).
- $\sigma_A$ é o desvio padrão da característica $x$ para a classe $A$.

Essa formulação nos permite lidar com variáveis contínuas de forma eficaz. O modelo Gaussiano é bastante útil quando temos dados numéricos, como idade ou renda, e as suposições de independência de características ainda se mantêm.

Agora que entendemos a teoria por trás do Naive Bayes Gaussiano, vamos aplicá-lo ao conjunto de dados do Titanic e ver como podemos prever a sobrevivência dos passageiros com base em características como idade, gênero e classe de viagem.


In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import joblib

In [ ]:

# Carregando o dataset
data = pd.read_csv('train.csv')

# Exibindo as primeiras linhas para entender a estrutura dos dados
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S



## Explorando o Dataset

O dataset contém diversas informações sobre os passageiros do Titanic, como:

- **Survived**: Se o passageiro sobreviveu (1) ou não (0).
- **Pclass**: A classe do passageiro (1ª, 2ª, 3ª).
- **Sex**: Gênero do passageiro.
- **Age**: Idade do passageiro.
- **SibSp**: Número de irmãos/cônjuges a bordo.
- **Parch**: Número de pais/filhos a bordo.
- **Fare**: Tarifa paga pelo passageiro.
- **Embarked**: Porto de embarque.

Vamos utilizar essas colunas para treinar nosso modelo de Naive Bayes.


In [ ]:

# Removendo colunas desnecessárias
data_cleaned = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Preenchendo valores ausentes
data_cleaned['Age'].fillna(data_cleaned['Age'].median(), inplace=True)
data_cleaned['Fare'].fillna(data_cleaned['Fare'].median(), inplace=True)
data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0], inplace=True)

# Codificando variáveis categóricas
label_encoder_sex = LabelEncoder()
label_encoder_embarked = LabelEncoder()
data_cleaned['Sex'] = label_encoder_sex.fit_transform(data_cleaned['Sex'])
data_cleaned['Embarked'] = label_encoder_embarked.fit_transform(data_cleaned['Embarked'])

# Exibindo os dados limpos
data_cleaned.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2



## Preparação dos Dados

Removemos as colunas irrelevantes para a predição e tratamos os valores ausentes. Além disso, codificamos variáveis categóricas como `Sex` e `Embarked` para que possam ser utilizadas pelo algoritmo de aprendizado de máquina.


In [ ]:

# Separando as features e o target
X = data_cleaned.drop(columns=['Survived'])
y = data_cleaned['Survived']

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Avaliando a acurácia no conjunto de teste
y_pred = nb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do Naive Bayes: {accuracy * 100:.2f}%")

Acurácia do Multinomial Naive Bayes: 77.65%



## Treinamento do Modelo

Após a preparação dos dados, treinamos o modelo de Naive Bayes. A acurácia obtida no conjunto de teste mostra o desempenho do modelo.


In [ ]:

# Salvando o modelo treinado
joblib.dump(nb_model, 'naive_bayes_titanic_model.pkl')


['naive_bayes_titanic_model.pkl']


## Salvando o Modelo

Aqui salvamos o modelo treinado em um arquivo, para que possamos reutilizá-lo sem precisar treiná-lo novamente.


In [ ]:

# Carregando o modelo salvo e fazendo previsões
loaded_model = joblib.load('naive_bayes_titanic_model.pkl')

# Função para fazer previsões com o modelo carregado
def predict_survival_with_loaded_model(pclass, sex, age, sibsp, parch, fare, embarked):
    # Codificando as variáveis categóricas
    sex_encoded = label_encoder_sex.transform([sex])[0]
    embarked_encoded = label_encoder_embarked.transform([embarked])[0]

    # Criando os dados de entrada no formato esperado pelo modelo
    input_data = [[pclass, sex_encoded, age, sibsp, parch, fare, embarked_encoded]]

    # Fazendo a predição com o modelo carregado
    prediction = loaded_model.predict(input_data)[0]

    # Retornando o resultado
    return 'Sobreviveu' if prediction == 1 else 'Não sobreviveu'

# Testando a função com o modelo carregado
predict_survival_with_loaded_model(3, 'female', 10, 1, 0, 0, 'C')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


'Sobreviveu'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd

# Carregue seu dataset
data = pd.read_csv('train.csv')

# Preparando os dados
data_cleaned = data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
data_cleaned['Age'].fillna(data_cleaned['Age'].median(), inplace=True)
data_cleaned['Fare'].fillna(data_cleaned['Fare'].median(), inplace=True)
data_cleaned['Embarked'].fillna(data_cleaned['Embarked'].mode()[0], inplace=True)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_cleaned['Sex'] = label_encoder.fit_transform(data_cleaned['Sex'])
data_cleaned['Embarked'] = label_encoder.fit_transform(data_cleaned['Embarked'])

# Separando os dados
X = data_cleaned.drop(columns=['Survived'])
y = data_cleaned['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertendo dados contínuos para inteiros
X_train_int = X_train.copy()
X_test_int = X_test.copy()
X_train_int['Age'] = X_train_int['Age'].astype(int)
X_train_int['Fare'] = X_train_int['Fare'].astype(int)
X_test_int['Age'] = X_test_int['Age'].astype(int)
X_test_int['Fare'] = X_test_int['Fare'].astype(int)

# Treinando o modelo Multinomial Naive Bayes
mnb_model = MultinomialNB()
mnb_model.fit(X_train_int, y_train)

# Fazendo previsões
y_pred_mnb = mnb_model.predict(X_test_int)

# Calculando a acurácia
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
print(f"Acurácia do Multinomial Naive Bayes: {accuracy_mnb * 100:.2f}%")


Acurácia do Multinomial Naive Bayes: 73.18%
